# Question Augmentation (Edge Expanding)

Users, who do not know the details of the underlying data, need help in constructing meaningful queries.   This service takes queries, and attempts to expand each edge using rules that maximize precision and recall.